In [1]:
%cd /content/drive/MyDrive/202201_bigdata_share

/content/drive/MyDrive/202201_bigdata_share


# **Chapter 4. A to Z 빅데이터 분석 실습**

## 4. 실습 A to Z - Census Income 

### (2) 사전준비

#### 442-01 : 패키지 불러오기 

In [2]:
import pandas as pd
import numpy as np
import warnings

442-02 : 데이터 로딩

In [121]:
X_test = pd.read_csv("dataset/census_X_test.csv")
X_train = pd.read_csv("dataset/census_X_train.csv")
y_train = pd.read_csv("dataset/census_y_train.csv")

### (3) 데이터 전처리

#### 443-01 : 데이터 정보 확인 

In [4]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24129 entries, 0 to 24128
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             24129 non-null  int64 
 1   workclass       24129 non-null  object
 2   education_num   24129 non-null  int64 
 3   marital_status  24129 non-null  object
 4   occupation      24129 non-null  object
 5   relationship    24129 non-null  object
 6   race            24129 non-null  object
 7   sex             24129 non-null  object
 8   capital_gain    24129 non-null  int64 
 9   capital_loss    24129 non-null  int64 
 10  hours_per_week  24129 non-null  int64 
 11  native_country  24129 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.2+ MB
None


443-02 : 데이터 통계량 확인

In [5]:
print(X_train.describe())

                age  education_num  capital_gain  capital_loss  hours_per_week
count  24129.000000   24129.000000  24129.000000  24129.000000    24129.000000
mean      38.421277      10.123337   1082.373575     88.112064       40.985205
std       13.131497       2.546004   7315.004017    404.452688       11.979781
min       17.000000       1.000000      0.000000      0.000000        1.000000
25%       28.000000       9.000000      0.000000      0.000000       40.000000
50%       37.000000      10.000000      0.000000      0.000000       40.000000
75%       47.000000      13.000000      0.000000      0.000000       45.000000
max       90.000000      16.000000  99999.000000   4356.000000       99.000000


#### 443-03 : capital_gain변수 통계량 확인

In [6]:
print(X_train['capital_gain'].quantile([q/20 for q in range(15, 21)]))

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64


#### 443-04 : capital_loss변수 통계량 확인

In [7]:
print(X_train['capital_loss'].quantile([q/20 for q in range(15, 21)]))

0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64


#### 443-05 : 파생변수 생성

In [122]:
X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0, 1, 0)
X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0, 1, 0)

#### 443-06 : 변수 구분 

In [123]:
# 불필요 변수
COL_DEL = ['capital_loss']
# 수치형 변수
COL_NUM = ['age', 'education_num', 'hours_per_week', 'capital_gain']
# 범주형 변수 
COL_CAT = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'capital_gain_yn']
# 종속 변수
COL_Y = ['target']

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

#### 443-07 : 수치형 변수 탐색

In [124]:
train_df = pd.concat([X_train, y_train], axis=1)

for _col in COL_NUM:
  print('-'*80)
  print(_col)
  print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')

--------------------------------------------------------------------------------
age
          count       mean        std   min   25%   50%   75%   max
target                                                             
0       18113.0  36.592668  13.464529  17.0  26.0  34.0  45.0  90.0
1        6016.0  43.926862  10.266862  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       18113.0   9.637056  2.413934  1.0   9.0   9.0  10.0  16.0
1        6016.0  11.587434  2.366362  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       18113.0  39.429912  11.983082  1.0  38.0  40

#### 443-08 : 범주형 변수 탐색

In [125]:
for _col in COL_CAT:
  print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')

          workclass    target
3      Self-emp-inc  0.560276
0       Federal-gov  0.387054
1         Local-gov  0.297329
4  Self-emp-not-inc  0.282270
5         State-gov  0.267522
2           Private  0.219120
6       Without-pay  0.000000

          marital_status    target
2     Married-civ-spouse  0.454747
1      Married-AF-spouse  0.437500
0               Divorced  0.109720
6                Widowed  0.095890
3  Married-spouse-absent  0.091525
5              Separated  0.067400
4          Never-married  0.048843

           occupation    target
3     Exec-managerial  0.484380
9      Prof-specialty  0.442897
10    Protective-serv  0.328330
12       Tech-support  0.309760
11              Sales  0.275528
2        Craft-repair  0.223352
13   Transport-moving  0.201122
1        Armed-Forces  0.142857
0        Adm-clerical  0.140404
6   Machine-op-inspct  0.120301
4     Farming-fishing  0.119441
5   Handlers-cleaners  0.065874
7       Other-service  0.042570
8     Priv-house-serv  0.00000

## (4) 데이터 모형 구축

#### 444-01 : 범주형 변수 - 레이블 인코딩

In [126]:
from sklearn.preprocessing import LabelEncoder

for _col in COL_CAT:
  le = LabelEncoder()
  le.fit(X_train[_col])
  X_train[_col] = le.transform(X_train[_col])
  X_test[_col] = le.transform(X_test[_col])

#### 444-02 : 훈련데이터 분할

In [127]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

#### 444-03 : 수치형 변수 - 데이터 스케일링

In [128]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

#### 444-04 : 로지스틱 회귀 학습

In [129]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=5000)
model_lr.fit(X_tr, y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=5000)

#### 444-05 : 랜덤포레스트 학습

In [130]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier()

#### 444-06 : XGBoost 학습

In [131]:
import xgboost as xgb
from xgboost import XGBClassifier

model_xgb = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
model_xgb.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

[0]	validation_0-auc:0.892958
Will train until validation_0-auc hasn't improved in 50 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10]	validation_0-auc:0.899215
[20]	validation_0-auc:0.90296
[30]	validation_0-auc:0.906043
[40]	validation_0-auc:0.907051
[50]	validation_0-auc:0.908547
[60]	validation_0-auc:0.909465
[70]	validation_0-auc:0.90976
[80]	validation_0-auc:0.909727
[90]	validation_0-auc:0.909801
[100]	validation_0-auc:0.90942
[110]	validation_0-auc:0.909362
[120]	validation_0-auc:0.909302
[130]	validation_0-auc:0.908876
Stopping. Best iteration:
[85]	validation_0-auc:0.909936



XGBClassifier(max_depth=10, n_estimators=1000)

#### 444-07 : 검증 데이터 예측

In [132]:
y_pred_lr = model_lr.predict_proba(X_val)
y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb = model_xgb.predict_proba(X_val)

#### 444-08 : 평가지표 계산

In [133]:
from sklearn.metrics import roc_auc_score

score_lr = roc_auc_score(y_val, y_pred_lr[:, 1])
score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb = roc_auc_score(y_val, y_pred_xgb[:, 1]) 

print(score_lr)
print(score_rf)
print(score_xgb)

0.8487068697449219
0.8813309958739322
0.9099359016839699


#### 444-09 : 변수 중요도 확인

In [150]:
pd.DataFrame({'feature': X_tr.columns, 'fi_rf': model_rf.feature_importances_, 'fi_xgb': model_xgb.feature_importances_})

,feature,fi_rf,fi_xgb
0,age,0.228210,0.028702
1,workclass,0.052229,0.025300
2,education_num,0.142200,0.092466
3,marital_status,0.082804,0.140668
4,occupation,0.095488,0.028395
5,relationship,0.115750,0.389192
6,race,0.017597,0.023426
7,sex,0.012085,0.021857
8,capital_gain,0.096852,0.198694
9,hours_per_week,0.112290,0.029209


#### 444-10 : 변수 제거

In [151]:
X_train = X_train.drop('capital_gain_yn', axis=1)
X_test = X_test.drop('capital_gain_yn', axis=1)

KeyError: ignored

#### 444-11 : 랜덤 포레스트 하이퍼파라미터 튜닝

In [155]:
from sklearn.model_selection import GridSearchCV

params = { 
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_cv = GridSearchCV(estimator=model_rf, param_grid=params, cv=5)
rf_cv.fit(X_train, y_train.values.ravel()) # 약 10분 소요

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

#### 444-12 : CV 결과 확인 1

In [158]:
print(pd.DataFrame(rf_cv.cv_results_).head())

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.614654      0.014644         0.047466        0.010361   
1       1.021098      0.311534         0.061359        0.009839   
2       1.551375      0.045204         0.111706        0.003587   
3       0.412004      0.007714         0.032109        0.002396   
4       0.906183      0.202203         0.068138        0.016301   

  param_max_depth param_min_samples_leaf param_min_samples_split  \
0               5                      1                       2   
1               5                      1                       2   
2               5                      1                       2   
3               5                      1                       5   
4               5                      1                       5   

  param_n_estimators                                             params  \
0                 50  {'max_depth': 5, 'min_samples_leaf': 1, 'min_s...   
1                100  {'max_depth': 5,

#### 444-13 : CV 결과 확인 2

In [159]:
rf_cv.best_params_

{'max_depth': 15,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

#### 444-14 : 랜덤포레스트 학습 및 결과 확인 

In [160]:
model_rf2 = RandomForestClassifier(n_estimators=200
                                   , max_depth=15
                                   , min_samples_leaf=2
                                   , min_samples_split=2)
model_rf2.fit(X_tr, y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:, 1])
print(score_rf2)

0.9070418938111021


#### 444-15 : XGBoost 하이퍼파라미터 튜닝

In [83]:
model_xgb2 = XGBClassifier(n_estimators=100)

grid_params = {'learning_rate': [0.05, 0.1],
          'max_depth': [5, 10], 
          'min_child_weight': [1, 3], 
          'colsample_bytree': [0.5, 0.8],
          'subsample': [0.5, 0.8]}

xgb_cv = GridSearchCV(model_xgb2, param_grid=grid_params, cv=3)
xgb_cv.fit(X_tr, y_tr.values.ravel())

GridSearchCV(cv=3, estimator=XGBClassifier(),
             param_grid={'colsample_bytree': [0.5, 0.8],
                         'learning_rate': [0.05, 0.1], 'max_depth': [5, 10],
                         'min_child_weight': [1, 3], 'subsample': [0.5, 0.8]})

#### 444-16 : CV 결과 확인 3

In [84]:
xgb_cv.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'subsample': 0.8}

#### 444-17 : XGBoost 학습 및 결과 확인

In [88]:
params = {'colsample_bytree': 0.5,
          'learning_rate': 0.1,
          'max_depth': 10,
          'min_child_weight': 1,
          'subsample': 0.8}

model_xgb3 = XGBClassifier(n_estimators=1000)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

print(model_xgb3.best_score)

XGBClassifier(colsample_bytree=0.5, max_depth=10, n_estimators=1000,
              subsample=0.8)

#### 444-18 : 결과 제출

In [ ]:
pred = model_xgb3.predict_proba(X_test)[:,1]
pd.DataFrame({'index': X_test.index, 'target': pred}).to_csv('003000000.csv', index=False)